In [ ]:
!git clone https://github.com/ToelUl/Thermal-Phase-Transition-MC-GPU-Simulation.git

!cp -r Thermal-Phase-Transition-MC-GPU-Simulation/gpu_mc ./

In [ ]:
import time
import matplotlib.pyplot as plt

from gpu_mc import PottsModel

import torch

if not torch.cuda.is_available():
    print("⚠️ GPU runtime is not enabled. Please go to 'Runtime' > 'Change runtime type' and set 'Hardware accelerator' to 'GPU'.")
else:
    print("✅ GPU runtime is enabled and ready!")

In [ ]:
L = 32

T = torch.linspace(0.5, 1.5, steps=32)

sampler_potts = PottsModel(
    L=L,
    T=T,
    q=3,
    n_chains=30,
    device=torch.device("cuda"),
    use_amp=True,
    pt_enabled=False,
    )

start = time.time()
samples_potts = sampler_potts(n_sweeps=1000, n_therm=5000, decorrelate=10)
end = time.time()
print(f"Elapsed time: {end - start:.2f} s")

In [ ]:
print(f"Samples shape: {samples_potts.shape}")

sampler_potts.spins = samples_potts

energy_potts = sampler_potts.compute_energy().mean(dim=1).cpu().numpy() / L**2
capacity_potts = sampler_potts.compute_heat_capacity().cpu().numpy()

plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(T.cpu().numpy(), energy_potts, label='Energy', marker='o')
plt.xlabel('Temperature')
plt.ylabel('Energy')
plt.title('Energy vs. Temperature')
plt.subplot(122)
plt.plot(T.cpu().numpy(), capacity_potts, label='Heat Capacity', marker='o')
plt.xlabel('Temperature')
plt.ylabel('Specific Heat Capacity')
plt.title('Specific Heat Capacity vs. Temperature')
plt.tight_layout()